In [ ]:
# default_exp core

# 00 Prodb API details

> `nbdev` Example function.

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

## Prodb API
Create pandas dataframe for manipulation

In [ ]:
#export
#hide
import pandas as pd
import numpy as np
import arrow
import os
from time import gmtime, strftime
#from prodb.core import generate_db, insert_row, utc_now, readable_df

In [ ]:
from Object import ClassName
class Pro(Object.pd):
    print('hi')

ModuleNotFoundError: No module named 'Object'

In [ ]:
class SubclassedSeries(pd.Series):
    @property
    def _constructor(self):
        return SubclassedSeries

    @property
    def _constructor_expanddim(self):
        return SubclassedDataFrame

class SubclassedDataFrame(pd.DataFrame):
    @property
    def _constructor(self):
        return SubclassedDataFrame

    @property
    def _constructor_sliced(self):
        return SubclassedSeries
    
df = SubclassedDataFrame(data)
df

,name,mood,message,time_utc
0,Sam,😊,hi,2021-11-07 22:02:49
1,Grant,😵,hello,2021-11-07 22:02:49


In [ ]:
ds.geo.center
(5.0, 10.0)
ds.geo.plot()

NameError: name 'ds' is not defined

In [ ]:
class MyDF(pd.DataFrame):
    @property
    def _constructor(self):
        return MyDF

data = {'name': ['Sam', 'Grant'],
                           'mood': ['😊', '😵'],
                           'message': ['hi', 'hello'],
                           'time_utc' : [arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
                                         arrow.utcnow().format('YYYY-MM-DD HH:mm:ss')]}

df = MyDF.from_dict(data)
df.head()

,name,mood,message,time_utc
0,Sam,😊,hi,2021-11-07 22:00:30
1,Grant,😵,hello,2021-11-07 22:00:30


In [ ]:
class Prodb(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        # use the __init__ method from DataFrame to ensure
        # that we're inheriting the correct behavior
        super(ExtendedDataFrame, self).__init__(*args, **kwargs)

    # this method is makes it so our methods return an instance
    # of ExtendedDataFrame, instead of a regular DataFrame
    @property
    def _constructor(self):
        return ExtendedDataFrame

    # now define a custom method!
    # note that `self` is a DataFrame
    def p(self):
        print()

In [ ]:
class Prodb(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        super(MyDF,  self).__init__(*args, **kwargs)

    @property
    def _constructor(self):
        return MyDF

    def s(self):
        print('This actually works!')
        
        
df = MyDF(data)
print(df)
df.my_custom_method()  # prints "This actually works!"

In [ ]:
class Prodb(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        super(MyDF,  self).__init__(*args, **kwargs)

    @property
    def _constructor(self):
        return MyDF

    def s(self):
        print('This actually works!')
        
        
df = MyDF(data)
print(df)
df.my_custom_method()  # prints "This actually works!"

    name mood message             time_utc
0    Sam    😊      hi  2021-11-07 22:02:49
1  Grant    😵   hello  2021-11-07 22:02:49
This actually works!


In [ ]:

class Prodb(pd.DataFrame):
    def __init__(self, *args, **kw):
        super(Prodb, self).__init__(*args, **kw)
        
    def pro(self):
        print(self.shape)
        

data = {'name': ['Sam', 'Grant'],
                           'mood': ['😊', '😵'],
                           'message': ['hi', 'hello'],
                           'time_utc' : [arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
                                         arrow.utcnow().format('YYYY-MM-DD HH:mm:ss')]}


df = Prodb(data)
df.pro()


(2, 4)


In [ ]:
class MyDF(pd.DataFrame):
    _attributes_ = "myattr1,myattr2"

    def __init__(self, *args, **kw):
        super(MyDF, self).__init__(*args, **kw)
        if len(args) == 1 and isinstance(args[0], MyDF):
            args[0]._copy_attrs(self)
            
    def s(self):
        print(self.df.shape)

mydf = MyDF(np.random.randn(3,4), columns=['A','B','C','D'])
mydf

,A,B,C,D
0,0.463719,0.669329,1.975601,1.037729
1,-1.440767,0.889676,1.313284,3.062662
2,-0.501210,-0.192705,-0.367762,0.833936


In [ ]:
b.f()

Base.f()


In [ ]:
class MyDF(pd.DataFrame):
    _attributes_ = "myattr1,myattr2"

    def __init__(self, *args, **kw):
        super(MyDF, self).__init__(*args, **kw)
        if len(args) == 1 and isinstance(args[0], MyDF):
            args[0]._copy_attrs(self)

    def _copy_attrs(self, df):
        for attr in self._attributes_.split(","):
            df.__dict__[attr] = getattr(self, attr, None)

    @property
    def _constructor(self):
        def f(*args, **kw):
            df = MyDF(*args, **kw)
            self._copy_attrs(df)
            return df
        return f

mydf = MyDF(np.random.randn(3,4), columns=['A','B','C','D'])

In [ ]:
class pdb(pd):
    
    def __init__(self, name):
        self.name = name
    
    def summary(self):
        print(self.name)
        
prodb = pdb(pd, name='luke')

TypeError: module() takes at most 2 arguments (3 given)

In [ ]:
prodb = pd

df = prodb.DataFrame({'name': ['Sam', 'Grant'],
                           'mood': ['😊', '😵'],
                           'message': ['hi', 'hello'],
                           'time_utc' : [arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
                                         arrow.utcnow().format('YYYY-MM-DD HH:mm:ss')]})
df

,name,mood,message,time_utc
0,Sam,😊,hi,2021-11-07 21:32:46
1,Grant,😵,hello,2021-11-07 21:32:46


## 1.0 Generate Db
Initialise database with `generate_db` call. 

In [ ]:
# export

def generate_db(dbpath='db.csv', 
                cols=['_c1', '_c2', '_c3'],
                verbose=True,
                prefill=False):
    """Create example database"""
    if prefill:
        df = pd.DataFrame({'name': ['Sam', 'Grant'],
                           'mood': ['😊', '😵'],
                           'message': ['hi', 'hello'],
                           'time_utc' : [arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
                                         arrow.utcnow().format('YYYY-MM-DD HH:mm:ss')]})
    else:
        df = pd.DataFrame(columns=cols)
    df.to_csv(dbpath, index=None)
    if verbose: print(f'✓💾 {dbpath} ({os.path.getsize(dbpath)} kb)')
    return df


Generate empty `db` object with arbitary columns.

In [ ]:
df = generate_db(cols=['c1', 'c2', 'c3'])
df.head()

✓💾 db.csv (10 kb)


,c1,c2,c3


Generate `db` with some example data.

In [ ]:
df = generate_db(prefill=True)
df.head()

✓💾 db.csv (99 kb)


,name,mood,message,time_utc
0,Sam,😊,hi,2021-11-06 19:09:50
1,Grant,😵,hello,2021-11-06 19:09:50


## 2.0 Insert Row

In [ ]:
#export

def insert_row(df, data, dbpath='db.csv'):
    new_row = pd.Series(data)
    df = df.append(new_row, ignore_index=True)
    df.to_csv(dbpath, index=None)
    return df

def insert_rows(df, data, dbpath='db.csv'):
    new_rows = pd.DataFrame.from_dict(data)
    df = df.append(new_rows, ignore_index=True)
    df.to_csv(dbpath, index=None)
    return df


### Example: Insert single row

In [ ]:
df = generate_db(cols='name mood message'.split())

# ================================================================ #

data = {'name':'George', 'mood': '👹', 'message':'hi'}
df = insert_row(df, data)

# ================================================================ #

display(df)


✓💾 db.csv (19 kb)


,name,mood,message
0,George,👹,hi


### Example: Insert multiple rows
Note: `prodb` allows additional columns to be added after the database has been initialised.   
 
i.e. the `['name', 'mood', 'message']` columns that initialise the db, play nice when the additional `time_utc` dict is passed in. NaNs are imputed for previous rows.

First, a helper function to clean up timestamp calls.

In [ ]:
# export
def utc_now():
    return arrow.utcnow().format('YYYY-MM-DD HH:mm:ss')

In [ ]:
df = generate_db(cols='name mood message'.split())

# ================================================================ #

data = {'name': ['Sam', 'Grant'],
        'mood': ['😊', '😵'],
        'message': ['hello from London, UK', 'hello from Christchurch, NZ'],
        'time_utc' : [utc_now(), utc_now()]}

df = insert_rows(df, data)

# ================================================================ #

display(df)

✓💾 db.csv (19 kb)


,name,mood,message,time_utc
0,Sam,😊,"hello from London, UK",2021-11-06 19:10:12
1,Grant,😵,"hello from Christchurch, NZ",2021-11-06 19:10:12


Add another entry to the bottom of the database using `insert_row()`.   

Note: Data columns from dictionary do not need to be in pre-defined order.

In [ ]:
data = {'time_utc' : utc_now(),
        'name': 'Luke', 
        'mood': '👹', 
        'message': 'hello from London, UK'}
df = insert_row(df, data)

display(df)

,name,mood,message,time_utc
0,Sam,😊,"hello from London, UK",2021-11-06 19:10:12
1,Grant,😵,"hello from Christchurch, NZ",2021-11-06 19:10:12
2,Luke,👹,"hello from London, UK",2021-11-06 19:10:18


In [ ]:
data = {'name':'Bill', 'mood': '👹', 'message':'hi', 'time_utc':arrow.utcnow().format('YYYY-MM-DD HH:mm:ss')}
df = insert_row(df, data)
display(df)

,name,mood,message,time_utc
0,Sam,😊,"hello from London, UK",2021-11-06 19:10:12
1,Grant,😵,"hello from Christchurch, NZ",2021-11-06 19:10:12
2,Luke,👹,"hello from London, UK",2021-11-06 19:10:18
3,Bill,👹,hi,2021-11-06 19:10:19


In [ ]:
#export
def readable_df(df, max_rows=8, col_name='human_time'):
    if 'time_utc' in df.columns:
        df[col_name] = df.time_utc.apply(lambda x: arrow.get(x).humanize())
    return df.tail(8)

In [ ]:
data = {'name':'Luke', 
        'mood': '😊', 
        'message': 'hello, from UK',
        'time_utc': utc_now()}
df = insert_row(df, data)
readable_df(df, max_rows=10)

,name,mood,message,time_utc,human_time
0,Sam,😊,"hello from London, UK",2021-11-06 19:10:12,just now
1,Grant,😵,"hello from Christchurch, NZ",2021-11-06 19:10:12,just now
2,Luke,👹,"hello from London, UK",2021-11-06 19:10:18,just now
3,Bill,👹,hi,2021-11-06 19:10:19,just now
4,Luke,😊,"hello, from UK",2021-11-06 19:10:20,just now
